# Plot simulation results

In [1]:
library(ggplot2)
library(dplyr)
library(ggsci)
library(ggpubr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
perf<-readRDS("../results/performance_metrics.RDS") %>% select(-evalmod_out)

### Functions

In [14]:
min(perf$aucs,na.rm = T
   )

[1] 0.4150053

## Performance grids

In [62]:
methods<-unique(perf$method)
batches<-unique(perf$batch_effect)
for(method in methods){
    for(batch in batches){
        plt.data<-perf %>%
                     filter(batch_effect==batch, method==method) %>%
                     group_by(dPSI_th,dLogPSI_th) %>% 
                     summarize(mean_auc=mean(aucs),mean_num_pos=mean(num_pos), mean_num_neg=mean(num_neg)) 
        p<-plot.performance.grid(data = plt.data,bes = batch, method = method)
        fout<-paste0("../results/figures_v2/",method,".bes_",batch,".performance.pdf")
        ggsave(p,filename = fout,device = "pdf",height=4,width=3.5,units="in")
    }
}

`summarise()` has grouped output by 'dPSI_th'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'dPSI_th'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'dPSI_th'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'dPSI_th'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'dPSI_th'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'dPSI_th'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'dPSI_th'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'dPSI_th'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'dPSI_th'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'dPSI_th'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'dPSI_th'. You can overr

In [77]:
methods

[1] "trex.glm"    "trex_nb.glm" "rmats"       "suppa2"

## Differential performance grids

In [57]:
trex<-perf %>%
      filter(method=="trex.glm") %>%
      group_by(batch_effect,dPSI_th,dLogPSI_th) %>% 
      summarize(mean_auc=mean(aucs),
                mean_num_pos=mean(num_pos),
                mean_num_neg=mean(num_neg)) 
rmats<-perf %>%
      filter(method=="rmats") %>%
      group_by(method,batch_effect,dPSI_th,dLogPSI_th) %>% 
      summarize(mean_auc=mean(aucs),
                mean_num_pos=mean(num_pos),
                mean_num_neg=mean(num_neg)) %>%
      left_join(.,trex,by=c("batch_effect","dPSI_th","dLogPSI_th")) %>%
      mutate(mean_auc=mean_auc.y-mean_auc.x) # TRex - method
suppa<-perf %>%
      filter(method=="suppa2") %>%
      group_by(method,batch_effect,dPSI_th,dLogPSI_th) %>% 
      summarize(mean_auc=mean(aucs),
                mean_num_pos=mean(num_pos),
                mean_num_neg=mean(num_neg)) %>%
      left_join(.,trex,by=c("batch_effect","dPSI_th","dLogPSI_th")) %>%
      mutate(mean_auc=mean_auc.y-mean_auc.x) # TRex - method

`summarise()` has grouped output by 'batch_effect', 'dPSI_th'. You can override
using the `.groups` argument.
`summarise()` has grouped output by 'method', 'batch_effect', 'dPSI_th'. You
can override using the `.groups` argument.
`summarise()` has grouped output by 'method', 'batch_effect', 'dPSI_th'. You
can override using the `.groups` argument.


In [73]:
col.pal <- colorRampPalette(colors = c("#0000FF","#FFFFFF","#FF0000"))
for(method in methods[methods!="trex.glm"]){
    for(batch in batches){
        p<-plot.performance.grid(data = rmats,bes = 0, method = "rmats")+
          scale_fill_gradient2(low="#0000FF",mid="#FFFFFF",high="#FF0000",midpoint = 0,na.value = "#E4E5E5",limits=c(-0.1,0.25))+
          labs(fill=paste0("dAUROC TRex-",method))
            fout<-paste0("../results/figures_v2/difperformance.trex_vs_",method,".bes_",batch,".pdf")
            ggsave(p,filename = fout,device = "pdf",height=4,width=3.5,units="in")
    }
}

Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Scale for fill is already present.
Adding another scale

## Perfomance lines

In [151]:
dpsi<-0.25
dlpsi<-1.75
plt.data<-perf %>% 
          filter(dPSI_th==dpsi,dLogPSI_th==dlpsi,method!="trex_nb.glm") %>% 
          mutate(method=factor(method,levels=c("trex.glm","rmats","suppa2")))

p<-ggplot(plt.data,aes(x=batch_effect,y=aucs,fill=method,group=interaction(batch_effect,method)))+
    geom_boxplot(position="identity",alpha=0.1,coef=0)+
    geom_jitter(position = "identity",pch=21,size=3,show.legend = T)+
    theme_pubr()+
    scale_x_continuous(breaks = unique(plt.data$batch_effect))+
    scale_y_continuous(limits = c(0.6,1))+
    scale_fill_jco()

fout<-paste0("../results/figures_v2/performance.all_methods.by_bes.pdf")
        ggsave(p,filename = fout,device = "pdf",height=4,width=3.8,units="in")

## Plot effect sizes